In [1]:
import numpy as np
import pandas as pd
import os
from tqdm import tqdm
import torch
import torch.nn as nn

# from train_evaluate import NMR_prediction

import torch.nn.functional as F
import dgl
import dgl.nn as dglnn
import torch.nn as nn
from dgl import AddSelfLoop

from model_2d.NMR_gcn import NMR_GCN

from train_evaluate_2d import NMR_prediction

from preprocess.create_node_embedding.node_embeddings_godess import create_node_embeddings

from preprocess.build_adjaency_matrix.create_adjaency_matrix_godess import build_adjacency_matrix

from preprocess.create_graph.create_graph_data_godess import create_graph

# from dgl import save_graphs, load_graphs
# from tqdm import tqdm
from dgl import save_graphs, load_graphs


In [2]:
threshold_carbon=1.65

threshold_hydrogen=1.18

threshold_general=1.5

threshold_interaction=5.0

data_dir = 'godess/data_reformulate/'

In [3]:
# B = build_adjacency_matrix(labeled_pdb_dir = data_dir,
#                            threshold_carbon=threshold_carbon,
#                            threshold_hydrogen=threshold_hydrogen, 
#                            threshold_general=threshold_general, 
#                            threshold_interaction=threshold_interaction)
# B.calculate_all_matrix()

In [4]:
# num_test = 231
# Create = create_graph(data_dir=data_dir,
#                       out_atom_name_embed='merged_embed/atom_name_embed.csv',
#                       out_monosaccharide_simple_embed='merged_embed/monosaccharide_embed.csv',
#                       out_bound_AB_embed='merged_embed/ab_embed.csv',
#                       out_DL_embed='merged_embed/dl_embed.csv',
#                       out_PF_embed='merged_embed/pf_embed.csv',
#                       num_test=num_test)

# g, test_index = Create.create_all_graph()

--------------------------loading NMR Graph-------------------------------


100%|███████████████████████████████████████| 2310/2310 [02:39<00:00, 14.44it/s]


In [5]:
# save_graphs('godess/graph_data/data.bin', g)

g = load_graphs('godess/graph_data/data.bin')[0][0]

In [6]:
g.ndata['z']
g.ndata['train_carbon_mask']

tensor([False, False,  True,  ..., False, False, False])

In [7]:
num_epoch = 5000
lr = 1e-3

atom_dim = 256
mono_dim = 128
ab_dim = 64
dl_dim = 64
pf_dim = 64

In [8]:
in_size = atom_dim + mono_dim + ab_dim + dl_dim + pf_dim
# in_size = atom_dim

hidden_size_1 = int(in_size / 2)
hidden_size_2 = 256
hidden_size_3 = 128
hidden_size_4 = 32

In [9]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

g = g.to(device)

features = g.ndata['z']

labels = g.ndata['y']

masks = g.ndata['train_carbon_mask'], g.ndata['test_carbon_mask']

# masks = g.ndata['train_hydrogen_mask'], g.ndata['test_hydrogen_mask']

model = NMR_GCN(in_size=in_size, hid_size=[hidden_size_1, hidden_size_2,
                                       hidden_size_3, hidden_size_4], out_size=1).to(device)

NMR_prediction = NMR_prediction(results_dir='transfer_learning_results/2D_godess/training_carbon.csv',
                                results_dir_test = 'transfer_learning_results/2D_godess/testing_carbon.csv',
                                model_dir='transfer_learning_results/2D_godess/Model_Godess_carbon.pt',
                                num_epoch = num_epoch, 
                                lr = lr)

print("Training...")

NMR_prediction.train(g, features, labels, masks, model)

# test the model
print("Testing...")
saved_model = NMR_GCN(in_size=in_size, hid_size=[hidden_size_1, hidden_size_2,
                                                 hidden_size_3, hidden_size_4], out_size=1).to(device)
saved_model.load_state_dict(torch.load(NMR_prediction.model_dir))

acc1 = NMR_prediction.evaluate(g, features, labels, masks[0], saved_model, save_train=True, save_test = False)

acc2 = NMR_prediction.evaluate(g, features, labels, masks[1], saved_model, save_train=False, save_test = True)


print("train RMSE {:.4f}".format(acc1))
print("test RMSE {:.4f}".format(acc2))

Training...


  0%|                                        | 1/5000 [00:00<1:07:14,  1.24it/s]

8476
79830
Epoch 00000 | Loss 6670.8711 | train_RMSE 80.7612 | test_RMSE 81.0545 
8476
79830
Epoch 00001 | Loss 6522.3345 | train_RMSE 79.6151 | test_RMSE 79.9147 
8476


  0%|                                          | 5/5000 [00:01<15:14,  5.46it/s]

79830
Epoch 00002 | Loss 6338.7266 | train_RMSE 77.8917 | test_RMSE 78.2012 
8476
79830
Epoch 00003 | Loss 6067.0322 | train_RMSE 75.3542 | test_RMSE 75.6786 
8476
79830
Epoch 00004 | Loss 5678.4849 | train_RMSE 71.7238 | test_RMSE 72.0703 


...

100%|██████████████████████████████████████▉| 4997/5000 [07:48<00:00, 10.67it/s]

8476
79830
Epoch 04995 | Loss 13.4443 | train_RMSE 1.9896 | test_RMSE 2.0147 
8476
79830
Epoch 04996 | Loss 13.3266 | train_RMSE 1.8892 | test_RMSE 1.9236 
8476


100%|███████████████████████████████████████| 5000/5000 [07:48<00:00, 10.68it/s]

79830
Epoch 04997 | Loss 13.0505 | train_RMSE 1.9262 | test_RMSE 1.9503 
8476
79830
Epoch 04998 | Loss 12.9088 | train_RMSE 1.8477 | test_RMSE 1.8797 
8476
79830
Epoch 04999 | Loss 13.0446 | train_RMSE 1.8716 | test_RMSE 1.8950 
best loss: 12.621841430664062
Testing...


79830
8476
train RMSE 1.8340
test RMSE 1.8591
